## Grupo 4 - Laboratorio 2 - R

Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/>


### 1. Sample splitting to evaluate the performance of prediction rules to a fellow student and show how to use it on the wage data.


Sample siplitting or data sampling it is a statistical analysis technique. This consist in the division of the sample (randomly) into two subsamples: train and test. 
To use it on the wage data, we are going to use the train sub sample and make a simple regression of $Y1$(lwage) and $X1$(sex). Of this regression we find a $\beta$. We are going to use this $\beta$ to calculate a $Y$ estimated. 
Using the real $Y2$ of the test sub sample and the $Y$ estimated we just found, now we are capable of finding an accurate MSE and R-squared of the regression.


### 2. Replicate the OLS and lasso for wage prediction- Jannis Kueck


Goals of the paper:

* Predict wages  using various characteristics of workers, in this case we are only using the data of the workers who didnt go to college (shs,hsg).

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of not having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regressio


#### 2.1 Data Analysis

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$. 

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n=5150$.

We start by loading the data set.

In [1]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

In [2]:
install.packages("hdm")


package 'hdm' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'hdm'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Anaconda-Python\envs\new\Lib\R\library\00LOCK\hdm\doc\hdm.pdf  a C:\Anaconda-Python\envs\new\Lib\R\library\hdm\doc\hdm.pdf: Permission denied"Warning message:
"restored 'hdm'"


The downloaded binary packages are in
	C:\Users\Usuario\AppData\Local\Temp\RtmpKAf35D\downloaded_packages


In [3]:
vignette("hdm")


In [4]:
str(data)

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

We are going to focus on the subset of people who did not go to college (shs, hsg variables).

In [5]:
data <- data[data$shs==1 |data$hsg==1,]
dim(data)


[1] 1376   20

We can see the number of observarions has been reduced because we are working with a subsample.

In [6]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 1376 
Number of raw regressors: 18

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [7]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

In [8]:
## library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Experiencia","Some High School","High School Graduate","Some College","College Graduate","Midwest","South","West","ne","Experiencia")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
Log.Wage,2.7185624
Sex,0.3219477
Experiencia,0.0872093
Some.High.School,0.9127907
High.School.Graduate,0.0000000
Some.College,0.0000000
College.Graduate,0.0000000
Midwest,0.2863372
South,0.2914244
West,0.1984012


E.g., the share of female workers in our sample is ~32% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [9]:
print(tab, type="latex")

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% 
\begin{tabular}{rr}
  \hline
 & Sample mean \\ 
  \hline
Log.Wage & 2.72 \\ 
  Sex & 0.32 \\ 
  Experiencia & 0.09 \\ 
  Some.High.School & 0.91 \\ 
  High.School.Graduate & 0.00 \\ 
  Some.College & 0.00 \\ 
  College.Graduate & 0.00 \\ 
  Midwest & 0.29 \\ 
  South & 0.29 \\ 
  West & 0.20 \\ 
  ne & 0.22 \\ 
  Experiencia.1 & 17.19 \\ 
   \hline
\end{tabular}


##### Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors: (sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2).


2. Flexible Model:  $X$ consists on the next regressors: (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

### 2.2 Basic model

In [10]:
# 1. basic model
basic <- lwage~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we +occ2 + ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model



Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
  2.8330066   -0.0733094    0.0075742   -0.0811342           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA   -0.0431882   -0.1091620    0.0129620   -0.1961261   -0.0086113  
      occ24        occ25        occ26        occ27        occ28        occ29  
  0.0005078    0.2615289   -0.3510072   -0.1900342   -0.6616521   -0.3013316  
     occ210       occ211       occ212       occ213       occ214       occ215  
 -0.0576220   -0.4176903   -0.4663571   -0.4219896   -0.5527766   -0.4747648  
     occ216       occ217       occ218       occ219       occ220       occ221  
 -0.2381724   -0.3529422   -0.3976108   -0.1181885   -0.1053967   -0.1737437  
     occ222        ind23        ind24        ind25        ind26        ind27  
 -0.3479965    0.1742747    0.0504201    0.0585330    0.0348081    0.2379530

Number of regressors in the basic model: 51 


##### Note that the basic model consists of $51$ regressors.

### 2.3 Flexible model

In [11]:
# 2. flexible model
flex <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model



Call:
lm(formula = flex, data = data)

Coefficients:
  (Intercept)           exp1           exp2           exp3           exp4  
    1.601e+01     -3.411e+00      3.874e+01     -2.133e+01      7.337e+00  
          shs            hsg            scl            clg          occ22  
   -5.723e-01             NA             NA             NA     -2.441e+00  
        occ23          occ24          occ25          occ26          occ27  
   -4.590e+01      7.898e+00     -5.081e+01      1.803e+01     -9.997e-01  
        occ28          occ29         occ210         occ211         occ212  
   -1.024e+01     -2.064e+01     -3.857e+00      2.756e-01      1.765e+00  
       occ213         occ214         occ215         occ216         occ217  
   -2.087e+00     -8.440e-01     -1.059e+01     -1.232e-01     -4.549e+00  
       occ218         occ219         occ220         occ221         occ222  
   -3.445e-01     -6.630e+00     -2.868e+00     -1.239e+00     -1.932e+00  
        ind23          ind24      

Number of regressors in the flexible model: 979

Note that the flexible model consists of $979$ regressors.

### 2.4 Lasso Model

Try Lasso next to both models (basic and flexible)

In [12]:
library(hdm)
basiclasso <- lwage ~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we +occ2 + ind2)
regbasiclasso<- rlasso(basiclasso, data=data, penalty = list (gamma = 0.4/log(n)))
regbasiclasso
cat( "Number of regressors in the flexible model:",length(regbasiclasso$coef))


Warning message:
"package 'hdm' was built under R version 3.6.3"


Call:
rlasso.formula(formula = basiclasso, data = data, penalty = list(gamma = 0.4/log(n)))

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   2.701637    -0.095300     0.007798     0.000000     0.000000     0.000000  
        clg           mw           so           we        occ22        occ23  
   0.000000     0.000000    -0.107542     0.000000     0.000000     0.000000  
      occ24        occ25        occ26        occ27        occ28        occ29  
   0.000000     0.468441     0.000000     0.000000     0.000000     0.000000  
     occ210       occ211       occ212       occ213       occ214       occ215  
   0.000000     0.000000     0.000000    -0.249690    -0.325497    -0.302078  
     occ216       occ217       occ218       occ219       occ220       occ221  
   0.000000     0.000000     0.000000     0.000000     0.000000     0.000000  
     occ222        ind23        ind24        ind25        ind26        ind27  
   0.000000     0.00000

Number of regressors in the flexible model: 51

In [13]:
library(hdm)
flexlasso <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2
regflexlasso<- rlasso(flexlasso , data=data,  penalty = list (gamma = 0.4/log(n)))
regflexlasso
cat( "Number of regressors in the flexible model:",length(regflexlasso$coef))



Call:
rlasso.formula(formula = flexlasso, data = data, penalty = list(gamma = 0.4/log(n)))

Coefficients:
  (Intercept)           exp1           exp2           exp3           exp4  
     2.652191       0.004963       0.000000       0.000000       0.000000  
          shs            hsg            scl            clg          occ22  
     0.000000       0.000000       0.000000       0.000000       0.000000  
        occ23          occ24          occ25          occ26          occ27  
     0.000000       0.000000       0.000000       0.000000       0.000000  
        occ28          occ29         occ210         occ211         occ212  
     0.000000       0.000000       0.000000       0.000000       0.000000  
       occ213         occ214         occ215         occ216         occ217  
    -0.227707      -0.270191      -0.353170       0.000000       0.000000  
       occ218         occ219         occ220         occ221         occ222  
     0.000000       0.000000       0.000000       0.00000

Number of regressors in the flexible model: 979

Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [14]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)
sumbasiclasso<- summary(regbasiclasso)
sumflexlasso<- summary(regflexlasso)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.LB <- sumbasiclasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.LB, "\n")
R2.adjLB <- sumbasiclasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjLB, "\n")

R2.LF <- sumflexlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.LF, "\n")
R2.adjLF <- sumflexlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjLF, "\n")


Call:
rlasso.formula(formula = basiclasso, data = data, penalty = list(gamma = 0.4/log(n)))

Post-Lasso Estimation:  TRUE 

Total number of variables: 50
Number of selected variables: 8 

Residuals: 
     Min       1Q   Median       3Q      Max 
-1.30862 -0.28876 -0.02007  0.27897  3.44429 

            Estimate
(Intercept)    2.702
sex           -0.095
exp1           0.008
shs            0.000
hsg            0.000
scl            0.000
clg            0.000
mw             0.000
so            -0.108
we             0.000
occ22          0.000
occ23          0.000
occ24          0.000
occ25          0.468
occ26          0.000
occ27          0.000
occ28          0.000
occ29          0.000
occ210         0.000
occ211         0.000
occ212         0.000
occ213        -0.250
occ214        -0.325
occ215        -0.302
occ216         0.000
occ217         0.000
occ218         0.000
occ219         0.000
occ220         0.000
occ221         0.000
occ222         0.000
ind23          0.000
ind24        

In [15]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")

MSELB <-mean(sumbasiclasso$res^2)
cat("MSE for the lasso basic model with partialling out: ", MSELB, "\n")
pLB <- length(sumbasiclasso$coef)
MSE.adjLB <- (n/(n-pLB))*MSELB
cat("adjusted MSE for the lasso basic model : ", MSE.adjLB, "\n")

MSELF <-mean(sumflexlasso$res^2)
cat("MSE for the lasso flexible model: ", MSELF, "\n")
pLF <- length(sumflexlasso$coef)
MSE.adjLF <- (n/(n-pLF))*MSELF
cat("adjusted MSE for the lasso flexible model: ", MSE.adjLF, "\n")

MSE for the basic model:  0.2082191 
adjusted MSE for the basic model:  0.2157451 
MSE for the flexible model:  0.1252106 
adjusted MSE for the flexible model:  0.1953398 
MSE for the lasso basic model with partialling out:  0.2265784 
adjusted MSE for the lasso basic model :  0.2352995 
MSE for the lasso flexible model:  0.2256755 
adjusted MSE for the lasso flexible model:  0.7821901 


In [16]:
library(xtable)
table <- matrix(0, 4, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pLB,R2.LB,MSELB,R2.adjLB,MSE.adjLB)
table[4,1:5]   <- c(pLF,R2.LF,MSELF,R2.adjLF,MSE.adjLF)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg","lasso basic","lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% 
\begin{tabular}{rrrrrr}
  \hline
 & p & \$R\verb|^|2\_\{sample\}\$ & \$MSE\_\{sample\}\$ & \$R\verb|^|2\_\{adjusted\}\$ & \$MSE\_\{adjusted\}\$ \\ 
  \hline
basic reg & 48 & 0.18 & 0.21 & 0.15 & 0.22 \\ 
  flexible reg & 494 & 0.51 & 0.13 & 0.23 & 0.20 \\ 
  lasso basic & 51 & 0.11 & 0.23 & 0.10 & 0.24 \\ 
  lasso flex & 979 & 0.11 & 0.23 & 0.10 & 0.78 \\ 
   \hline
\end{tabular}


,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
basic reg,48,0.1802381,0.2082191,0.1512255,0.2157451
flexible reg,494,0.5070440,0.1252106,0.2315028,0.1953398
lasso basic,51,0.1079573,0.2265784,0.1027368,0.2352995
lasso flex,979,0.1115120,0.2256755,0.1017125,0.7821901


Considering all measures above, the flexible reg model performs so much better than the basic model.  

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

### 2.5 Data splitting

In [17]:
n

[1] 1376

In [18]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random <- sample(1:n, floor(n*4/5))
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample
dim(train)

[1] 1100   20

In [19]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   2.836027    -0.113776     0.008171    -0.045630           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA    -0.007887    -0.092965     0.027930    -0.198916    -0.014664  
      occ24        occ25        occ26        occ27        occ28        occ29  
   0.202306     0.242367     0.006158    -0.180164    -0.796769    -0.241348  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.015354    -0.399688    -0.506190    -0.446041    -0.575386    -0.450461  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.209308    -0.338949    -0.251586    -0.137541    -0.124913    -0.158960  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.350055     0.191507     0.036101     0.034002    -0.004881     0.21484

In [20]:
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

Warning message in predict.lm(regbasic, newdata = test):
"prediction from a rank-deficient fit may be misleading"

43       47      158      248      261      482      858      936 
2.872798 2.475568 2.909436 2.843801 2.706183 2.816292 3.030303 2.580716 
     954     1323     1490     1567     1630     1870     1964     2025 
2.801527 2.787998 3.062171 2.836898 2.326333 2.979703 2.285942 2.276830 
    2063     2362     2674     2925     3021     3065     3084     3170 
2.412655 2.711248 2.784423 2.636268 2.744333 2.677121 2.840804 3.074304 
    3385     3395     3581     3639     3715     3759     3770     3797 
2.828920 3.123457 2.388790 2.799951 3.210152 2.792775 2.264362 2.738699 
    3876     3917     4182     4183     4198     4222     4417     4610 
2.615367 2.817868 2.850550 2.798970 2.905476 2.129962 2.997718 2.490984 
    4671     4809     4869     4940     5069     5073     5180     5224 
2.297529 2.774117 2.733286 2.812450 2.890822 2.827126 2.760961 2.889827 
    5344     5598     5801     5848     6122     6236     6314     6533 
2.848974 3.119391 2.354723 2.940427 2.762220 2.724100 2.860109 2.901859 
    6844     6874     6881     7034     7137     7171     7309     7322 
2.265131 2.460612 2.263418 3.180935 2.501392 2.390470 2.795458 2.496359 
    7335     7381     7389     7929     7998     8136     8350     8471 
2.809982 2.997905 2.803136 2.738447 2.526222 2.960109 2.769129 2.708267 
    8478     8517     8572     8574     8578     8647     8802     8880 
2.664675 2.806314 2.703490 2.958472 2.820964 2.734791 2.906452 2.785470 
    9002     9111     9154     9360     9432     9456     9469     9488 
2.464904 2.993126 2.318446 2.779223 2.921167 2.636552 2.930964 2.704377 
    9504     9731     9770     9771     9852     9980    10208    10445 
2.421110 2.575738 2.505990 2.752206 2.809982 2.695625 2.825308 2.378906 
   10464    10709    10797    10905    11286    11490    11526    11542 
2.793640 2.795029 2.962288 2.826964 2.841088 2.933928 2.747153 2.839341 
   11858    12271    12361    12418    12439    12731    12753    12764 
3.083966 2.808405 2.480362 2.686605 2.844052 2.793640 2.986799 2.733305 
   12786    12867    12895    12980    13040    13105    13221    13354 
2.569409 2.760837 2.705152 2.948344 2.730090 2.968491 2.502339 2.901246 
   13358    13421    13603    13616    13728    13767    13858    13859 
2.482359 3.125308 2.160298 2.746005 2.666227 2.802424 2.752024 3.040440 
   14106    14134    14141    14189    14252    14326    14337    14683 
2.722042 2.407710 2.389074 2.669847 2.595581 2.843988 2.675921 2.824746 
   14692    14705    14748    14919    14943    14956    15015    15071 
2.876767 2.885334 2.712918 2.734495 2.450420 2.782129 2.572474 2.693666 
   15105    15193    15253    15295    15299    15385    15508    16328 
2.440899 2.458114 3.381992 2.756010 2.720160 3.187121 2.502988 2.643000 
   16365    16523    16841    17079    17152    17177    17249    17402 
2.535670 2.380429 2.893551 2.893823 2.723327 2.757586 2.886939 2.268831 
   18230    18257    18336    18375    18528    18558    18700    18846 
2.326511 2.729321 2.714885 2.658776 2.715157 2.387997 2.578446 2.620074 
   19056    19152    19181    19263    19615    19848    19987    19995 
2.845393 2.988053 2.934211 2.734891 2.547116 2.409090 2.303548 2.521727 
   20049    20062    20116    20198    20410    20423    20456    20653 
2.788741 2.619299 2.747839 2.702963 2.651368 2.899300 2.253588 2.704025 
   20777    20916    21030    21543    21626    21929    22080    22721 
2.479894 2.623468 2.858322 3.296119 2.892065 2.588540 2.680694 2.521727 
   22734    23053    23146    23353    23405    23517    23848    23991 
2.690374 2.718013 2.293828 2.776616 2.815392 2.704888 2.670543 2.800256 
   24374    24514    24649    25233    25417    25519    25767    25804 
3.132884 2.424292 2.569726 2.293828 1.983952 2.414019 2.866345 2.841260 
   25937    26067    26135    26331    26426    26620    26623    26696 
3.049481 2.589698 2.917757 3.106391 2.950831 2.512371 2.744083 2.750562 
   26804    26936    27200    27379    27395    27491    27541  

In [21]:
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)

Test MSE for the basic model:  0.2015247  Test R2 for the basic model:  0.06784807

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [22]:
# flexible model
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

Warning message in predict.lm(regflex, newdata = test):
"prediction from a rank-deficient fit may be misleading"

Test MSE for the flexible model:  22576.15  Test R2 for the flexible model:  -104424.9

The strange values can be explained because we have too many covariates for few observations.

In [23]:
length(y.test)

[1] 276

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [24]:
# basic model using lasso

# estimating the parameters
library(hdm)
reglassob <- rlasso(basic, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglassob<- predict(reglassob, newdata=test)
MSE.lassob <- sum((y.test-trainreglassob)^2)/length(y.test)
R2.lassob<- 1- MSE.lassob/var(y.test)


cat("Test MSE for the lasso on basic model: ", MSE.lassob, " ")

cat("Test R2 for the lasso basic model: ", R2.lassob)

Test MSE for the lasso on basic model:  0.2051704  Test R2 for the lasso basic model:  0.05098462

In [25]:

# flexible model using lasso

# estimating the parameters
library(hdm)
reglassof <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglassof<- predict(reglassof, newdata=test)
MSE.lassof <- sum((y.test-trainreglassof)^2)/length(y.test)
R2.lassof<- 1- MSE.lassof/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lassof, " ")

cat("Test R2 for the lasso flexible model: ", R2.lassof)

Test MSE for the lasso on flexible model:  0.2023553  Test R2 for the lasso flexible model:  0.06400605

Finally, let us summarize the results:

In [26]:
table2 <- matrix(0, 4,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lassob
table2[4,1]   <- MSE.lassof

table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lassob
table2[4,2]   <- R2.lassof

rownames(table2)<- c("basic reg","flexible reg","lasso basic regression","lasso flex regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

,$MSE_{test}$,$R^2_{test}$
basic reg,2.015247e-01,6.784807e-02
flexible reg,2.257615e+04,-1.044249e+05
lasso basic regression,2.051704e-01,5.098462e-02
lasso flex regression,2.023553e-01,6.400605e-02


In [27]:
print(tab2,type="latex") # type="latex" for printing table in LaTeX

% latex table generated in R 3.6.1 by xtable 1.8-4 package
% 
\begin{tabular}{rrr}
  \hline
 & \$MSE\_\{test\}\$ & \$R\verb|^|2\_\{test\}\$ \\ 
  \hline
basic reg & 0.202 & 0.068 \\ 
  flexible reg & 22576.151 & -104424.943 \\ 
  lasso basic regression & 0.205 & 0.051 \\ 
  lasso flex regression & 0.202 & 0.064 \\ 
   \hline
\end{tabular}


## 3. Partialling out using lasso

#### 3.1 Partialling out using lasso Matriz W=(exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)

In [28]:

library(sandwich)

#Example1
basic.y <- lwage ~  (exp1 + shs + hsg+ scl + clg + mw + so + we +occ2 + ind2) # model for Y
basic.d <- sex ~  (exp1 + shs + hsg+ scl + clg + mw + so + we +occ2 + ind2) # model for D

# partialling-out the linear effect of W from Y
basic.rY <- rlasso(basic.y, data=data)$res
# partialling-out the linear effect of W from D
basic.rD <- rlasso(basic.d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partial.fit1 <- lm(basic.rY~basic.rD)
partial.est1 <- summary(partial.fit1)$coef[2,1]

cat("Coefficient for D via partialling-out", partial.est1)

# standard error
HCV.coefs1 <- vcovHC(partial.fit1, type = 'HC')
partial.se1 <- sqrt(diag(HCV.coefs1))[2]

# confidence interval
confint(partial.fit1)[2,]


Coefficient for D via partialling-out -0.09065628

2.5 %      97.5 % 
-0.15242678 -0.02888578

When we use the lasso for partialling-out the basic regressions shows us that the gender gap is of $9.066\%$

#### 3.2 Partialling out using lasso Matrix W = (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2


In [29]:
#Example2
flex.y <- lwage ~   (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2 # model for Y 
flex.d <- sex ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2 # model for D

# partialling-out the linear effect of W from Y
flex.rY <- rlasso(flex.y, data=data)$res
# partialling-out the linear effect of W from D
flex.rD <- rlasso(flex.d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partial.fit2 <- lm(flex.rY~flex.rD)
partial.est2 <- summary(partial.fit2)$coef[2,1]

cat("Coefficient for D via partialling-out", partial.est2)

# standard error
HCV.coefs2 <- vcovHC(partial.fit2, type = 'HC')
partial.se <- sqrt(diag(HCV.coefs2))[2]

 #confidence interval
confint(partial.fit2)[2,]

Coefficient for D via partialling-out -0.08141371

2.5 %      97.5 % 
-0.14357666 -0.01925077

When we use the lasso for partialling-out the basic regressions shows us that the gender gap is of $8.14\%$